# 1. Tutorial on SAT Solving (DPLL)

This is a tutorial section on using the Davis-Putnam-Logemann-Loveland (DPLL) Algorithm to solve Boolean Satisfiability problems. Many codes in this tutorial can be found (together with references and acknowledgements) in the .py software version in the github: https://github.com/zhaoy37/SAT_Solver. The user can also follow the readme tutorial on the github. We leave any evaluations and solver comparisons to the python version part of the project in case the user is interested.

## Background on SAT solvers.

Imagine that you are provided with a boolean formula P and asked to determine if P is satisfiable. P is satisfiabile if there exists values for each variable such that P evaluates to true. For example, the formula $P = (x1 \wedge x2) \vee x3$ is satisfiable because there exists a model {x1 = 1, x2 = 1, x3 = 0} such that the the boolean formula evaluates to true. On the contrary, $P = (x1 \wedge \neg x1)$ is not satisfiable. Validity means that the formula evaluates to true for all models.

The implication of SAT solving is theoretically supported by the Cook-Levin Theorem, which states that Boolean SAT is NP-complete. Equivalently, any problem in NP can be reduced in polynomial time by a deterministic Turing Machine to a SAT problem. Therefore, many classical NP-complete problems in algorithms, such as graph coloring and sudoku, can be encoded into a SAT Problem representation and solved via a SAT solver.

## An Auxiliary Structure: A Logic Tree

The Davis-Putnam-Logemann-Loveland algorithm (DPLL) is a widely used algorithm in SAT solving. Its basic idea is recursive backtracking search of atom values that meet the satisfiability with some special heuristics. For the rest of the tutorial, we build a DPLL Solver from scratch.

We need to be able to represent SAT formulas in a data strucutre. To make things simple, we represent the formulas as lists. The strucuture is recursively determined. For instance, $P = (x1 \wedge x2) \vee x3$ can be represented as ["or", ["and", "x1", "x2"], "x3"]. The possible connectives include "or", "and", and "not".

To start with, to be able to quickly capture the properties of a SAT formula defined in list, we develop an auxiliary structure, which upon accepting the the list representation of the SAT formula, converts it to a tree representation in negative normal form (NNF), where the negations are directly in front of SAT atoms. We use the NNF logic tree to facilitate later procedure of the DPLL program.

In [1]:
class Logic:

    """
    Constructor

    The class takes in a list representation of the formula, and convert it to a
    logic tree representation. The converted data structure is a binary tree with
    connectives on the top. The left and right leaves are recursively defined binary
    trees or root atoms.
    """
    def __init__(self, formula):
        self.formula = formula
        self.left = None
        self.right = None
        self.value = self.__convert_formula_to_tree()
        self.leaves = self.load_leaves()
        self.pure_positives, self.pure_negatives = self.__find_pure_literals()


    """
    Perform an in-oder traversal of the tree for printing.
    This member function is majorly constructed for testing purposes.
    """
    def print_tree(self):
        if self.left is not None:
            self.left.print_tree()
        print(self.value)
        if self.right is not None:
            self.right.print_tree()

    """
    This member function finds the leaves of self. In this case, it finds the set
    of all SAT atoms.
    """
    def load_leaves(self):
        # Perform an in-order traversal.
        leaves = []
        if self.left is not None:
            leaves.extend(self.left.load_leaves())
        if (self.left is None) and (self.right is None):
            leaves.append(self.value)
        if self.right is not None:
            leaves.extend(self.right.load_leaves())
        leaves = [leaf for leaf in leaves if (leaf != "True" and leaf != "False")]
        return set(leaves)


    """
    This function finds the number of nodes of the tree.
    """
    def num_of_nodes(self):
        if self.left is None and self.right is None:
            return 1
        else:
            if self.left is None:
                return 1 + self.right.num_of_nodes()
            elif self.right is None:
                return 1 + self.left.num_of_nodes()
            else:
                return 1 + self.left.num_of_nodes() + self.right.num_of_nodes()


    """
    This method evaluates an assignment.
    The assignment is in the form of a dictionary mapping variables to boolean values.

    For instance, given that the formula P is ["and", "x1", "x2"] and an assignment
    {"x1" : 1, "x2" : 1}, the method should return true.
    """
    def evaluate(self, assignment):
        # Check if the assignment dictionary contains and only contains the leaves (excluding true and false) as the keys.
        if self.leaves != set(assignment.keys()):
            raise Exception("The given assignment is not valid")
        return self.__evaluate_assignment_kernel(assignment, self)


    """
    Private methods start here:
    """

    """
    This member function finds the literals of self.
    We use this function to facilitate the later DPLL algorithm.
    In the DPLL algorithm, we need to know the pure literals of a formula.
    We define the pure literals. The user may refer to the later portion of the material to
    see the definition of pure literals.
    """
    def __find_pure_literals(self):
        parents = dict()
        for leaf in self.leaves:
            parents[leaf] = []
        # Find the parents of the leaves.
        self.__find_parents_kernel(parents)
        # Find the pure positive and negative literals.
        pure_positives = [leaf for leaf in self.leaves if "not" not in parents[leaf]]
        pure_negatives = [leaf for leaf in self.leaves if (("not" in parents[leaf]) and len(parents[leaf]) == 1)]
        return pure_positives, pure_negatives


    """
    This member function is used in find_pure_literals.
    It finds the parents of the leaves.
    """
    def __find_parents_kernel(self, parents):
        if self.left is not None:
            if self.left.value in self.leaves:
                parents[self.left.value].append(self.value)
            self.left.__find_parents_kernel(parents)
        if self.right is not None:
            if self.right.value in self.leaves:
                parents[self.right.value].append(self.value)
            self.right.__find_parents_kernel(parents)


    """
    This method is the kernel to evaluate a potential assignment.
    The assignment is in the form of a dictionary mapping variables to boolean values.
    """
    def __evaluate_assignment_kernel(self, assignment, tree):
        if (tree.left is None) and (tree.right is None):
            if tree.value == "True":
                return True
            elif tree.value == "False":
                return False
            return assignment[tree.value]
        else:
            if tree.value == "not":
                return (not self.__evaluate_assignment_kernel(assignment, tree.right))
            elif tree.value == "and":
                return (self.__evaluate_assignment_kernel(assignment, tree.left) and
                        self.__evaluate_assignment_kernel(assignment, tree.right))
            else:
                return (self.__evaluate_assignment_kernel(assignment, tree.left) or
                        self.__evaluate_assignment_kernel(assignment, tree.right))

    """
    Convert parsed logic to a tree in NNF representation recursively.
    """
    def __convert_formula_to_tree(self):
        if not isinstance(self.formula, list):
            return self.formula
        else:
            if self.formula[0] == "not":
                if isinstance(self.formula[1], list) and self.formula[1][0] == "and":
                    self.left = Logic(["not", self.formula[1][1]])
                    self.right = Logic(["not", self.formula[1][2]])
                    return "or"
                elif isinstance(self.formula[1], list) and self.formula[1][0] == "or":
                    self.left = Logic(["not", self.formula[1][1]])
                    self.right = Logic(["not", self.formula[1][2]])
                    return "and"
                elif isinstance(self.formula[1], list) and self.formula[1][0] == "not":
                    equivalence = Logic(self.formula[1][1])
                    self.left = equivalence.left
                    self.right = equivalence.right
                    return equivalence.value
                else:
                    self.right = Logic(self.formula[1])
                    return self.formula[0]
            else:
                self.left = Logic(self.formula[1])
                self.right = Logic(self.formula[2])
                return self.formula[0]

Let's visualize one example: Here, the input formula is: $P = (\neg (x1 \wedge x2)\wedge x1)$. The list representation is: ["and", ["not", ["and", "x1", "x2"]], "x1"]. Using the Logic class, we convert the formula to a tree structure representing the equivalent formula in NNF: $P = (\neg x1 \vee \neg x2) \wedge x1$, as shown by the print_tree method, which prints the NNF representation in order.

In [2]:
logic = Logic(["and", ["not", ["and", "x1", "x2"]], "x1"])
logic.print_tree()

not
x1
or
not
x2
and
x1


## The DPLL Solver

The Davis-Putnam-Logemann-Loveland algorithm (DPLL) is a widely used algorithm in SAT solving. Its basic idea is recursive backtracking search of atom values that meet the satisfiability with some special heuristics.

Now, we briefly discuss the heuristics used in DPLL: There are two parts of the search algorithm in DPLL. In the recursive backtracking search, the algorithm first decides an assignment. Then, it deduces the assignment by substituting the atoms in the SAT formula with the assignment. If the valuation returns False, the algorithm backtracks to try another assignment if such new assignment is possible. If the algorithm runs out of assignments, the algorithm returns UNSAT. The heuristics in DPLL revolve around heuristics in deciding and deducing.

One heuristic in assigning is **Early termination**, which refers to terminating the algorithm if any realization of a variable leads to True or False of a SAT formula regardless the choice of any other variables. If the algorithmn returns False in the early termination, faster backtracking is encouraged. If the algorithm returns True in the early termination, on the other hand, faster path to a solution is found. To give a concrete example, $(A \vee B) \wedge (A \vee \neg C)$ is satisfied by {A = True}.

One heuristic for deciding assignments is **Pure literals**, which states that if all occurrences of a symbol in the clause (assuming a negation normal form) have the same sign accross the clause, we can guess that the symbol is True (if the symbol is positive) or False (if the symbol is negative). This can be done with the assumption that any "not" nodes far away from the leaf nodes are transformed to become the parents of the leaf nodes using the rules of De Morgan's Law and double negations (i.e. the SAT formula is in NNF). For instance, in $(A \vee B) \wedge (A \vee \neg C) \wedge (C \vee \neg B)$, the symbol A is pure and positive. By the heuristic, we should prioritize the decision of setting A to True over setting A to False in the search algorithm.

Another heuristic in assigning is **Unit Clauses**, which states that for any clause left with a single literal, we can simplify the clause by realizing that symbol. More broadly speaking, assignments to variables can lead to simplifications. For instance, in the case of {A = False} with $(A \vee B) \wedge (A \vee \neg C)$, the clause can be simplified to $(B) \wedge (\neg C)$
.

The above heuristics are what we implement in the DPLL solver. There are some more advanced heuristics that can increase the efficiency of the solver that we do not cover in our implementation.

Now, we begin with our function to simplify a Boolean formula following necessary heuristics.

In [3]:
def simplify(target, partial_assignment):
    """
    This function simplifies a formula given an assignment.

    target: the formula.
    partial_assignment: A possible assignment. An instance for the formula
    ["and", "x1", "x2"] is {"x1" : 1}.
    """
    if not isinstance(target, list):
        # This is the case of a single variable as the target formula.
        if target in partial_assignment:
            if partial_assignment[target]:
                return "True"
            else:
                return "False"
        else:
            return target
    else:
        if target[0] == "or":
            left_simplified = simplify(target[1], partial_assignment)
            right_simplified = simplify(target[2], partial_assignment)
            # Perform simplification.
            # Early termination:
            if left_simplified == "True" or right_simplified == "True":
                return "True"
            # Unit clause:
            elif left_simplified == "False":
                return right_simplified
            elif right_simplified == "False":
                return left_simplified
            else:
                return ["or", left_simplified, right_simplified]
        elif target[0] == "and":
            left_simplified = simplify(target[1], partial_assignment)
            right_simplified = simplify(target[2], partial_assignment)
            # Perform simplification.
            # Early termination:
            if left_simplified == "False" or right_simplified == "False":
                return "False"
            # Unit clause:
            if left_simplified == "True":
                return right_simplified
            elif right_simplified == "True":
                return left_simplified
            else:
                return ["and", left_simplified, right_simplified]
        else:
            right_simplified = simplify(target[1], partial_assignment)
            if right_simplified == "False":
                return "True"
            elif right_simplified == "True":
                return "False"
            else:
                return ["not", right_simplified]

Let's try some examples to see the heuristics we mentioned earlier.

In [4]:
# Early termination:
# (A or B) and (A or not C)
target = ["and", ["or", "x0", "x1"], ["or", "x0", ["not", "x2"]]]
print(simplify(target, {"x0" : 1}))

True


In [5]:
# Unit clause:
# (A or B) and (A or not C)
print(simplify(target, {"x0" : 0}))

['and', 'x1', ['not', 'x2']]


Now, let's look into the recursive backtracking part. We only focus on the solver that requires a single solution in our tutorial. For more comprehensive version, feel free to look into our python implementation.

In [6]:
# This is the entry point of our solver.
def solve_kernel_with_heuristic(target, variable_list, cur_assignment, pure_positives, pure_negatives, solutions):
    """
    This is the kernel for the solve function. It implements recursive backtracking.

    target: The SAT formula to be solved.
    variable_list: a list of SAT atoms.
    cur_assignment: We keep track of our current assignment in the recursive backtracking here.
    pure_positives: a list of SAT atoms that are pure positives.
    pure_negatives: a list of SAT atoms that are pure negatives.
    solutions: We keep track of our solution(s) in this data structure along the recursive backtracking process.
    """
    if len(cur_assignment.keys()) >= len(variable_list):
        # Base case: Reaches the end of assignment.
        return False
    else:
        index = len(cur_assignment.keys())
        variable = variable_list[index]
        # Perform the heuristic on pure literals.
        if variable in pure_positives:
            return solve_single(index, variable, target, variable_list, cur_assignment, pure_positives,
                                pure_negatives, solutions, [1, 0])
        else:
            return solve_single(index, variable, target, variable_list, cur_assignment, pure_positives,
                                pure_negatives, solutions, [0, 1])


# Now, we check out the solving procedure.
def solve_single(index, variable, target, variable_list, cur_assignment, pure_positives, pure_negatives,
                 solutions, ordering):
    """
    This function handles the case with solving for one solution only.

    cur_assignment: We keep track of our current assignment in the recursive backtracking here.
    """
    # Assign ordering[0] to the variable and try to simplify.
    new_assignment = cur_assignment.copy()
    new_assignment[variable] = ordering[0]
    simplified = simplify(target, new_assignment)
    # Case True following assigning ordering[0] to the variable: Return the correct answer.
    if simplified == "True":
        # Complete assignment.
        for i in range(index + 1, len(variable_list)):
            new_assignment[variable_list[i]] = 1
        solutions.append(new_assignment)
        return True
    # Case False following assigning ordering[0] to the variable: Try switch to ordering[1] and re-evaluate.
    elif simplified == "False":
        new_assignment[variable] = ordering[1]
        simplified = simplify(target, new_assignment)
        # If the solver attains true, return the correct answer.
        return further_search(index, simplified, variable_list, new_assignment, pure_positives, pure_negatives,
                              solutions)
    else:
        # The solver is inconclusive after assigning ordering[0] to the variable. Try to further simplify the formula.
        if solve_kernel_with_heuristic(simplified, variable_list, new_assignment, pure_positives,
                                       pure_negatives, solutions):
            return True
        else:
            # In this case, the solver resolves to false/inconclusive for assigning ordering[0] to the variable.
            new_assignment[variable] = ordering[1]
            simplified = simplify(target, new_assignment)
            return further_search(index, simplified, variable_list, new_assignment, pure_positives, pure_negatives,
                                  solutions)


# We require a further search step, which we present here:
def further_search(index, simplified, variable_list, new_assignment, pure_positives,
                   pure_negatives, solutions):
    """
    This function is used in solve_single for further searching and serves to help clean the codes.
    """
    if simplified == "True":
        # Complete assignment.
        for i in range(index + 1, len(variable_list)):
            new_assignment[variable_list[i]] = 1
        solutions.append(new_assignment)
        return True
    elif simplified == "False":
        # The assignment must be false.
        return False
    else:
        return solve_kernel_with_heuristic(simplified, variable_list, new_assignment,
                                           pure_positives, pure_negatives, solutions)

Now, let's write a function to initiate the kernel.

In [7]:
def solve(tree):
    """
    This is the main entry point for solving the formula using DPLL.

    The input is a SAT formula in its tree representation.
    """
    # Call the recursive backtracking kernel.
    variable_list = list(tree.leaves)
    solutions = []
    # Tree Heuristic automatically enabled in this branch.
    cur_assignment = dict()
    target = tree.formula
    solve_kernel_with_heuristic(target, variable_list, cur_assignment, tree.pure_positives, tree.pure_negatives, solutions)
    # Detect pure literals if assignment_heuristic_enabled: To be implemented later
    if len(solutions) == 0:
            return "UNSAT"
    else:
        return solutions[0]

Let's try some examples:

In [8]:
# Example 1: x1 and x2
formula = ["and", "x1", "x2"]
tree = Logic(formula)
solve(tree)

{'x2': 1, 'x1': 1}

In [9]:
# Example 2: x1 and (not (x2 or x1))
formula = ["and", "x1", ["not", ["or", "x2", "x1"]]]
tree = Logic(formula)
solve(tree)

'UNSAT'

In [10]:
# Example 3: x1
formula = "x1"
tree = Logic(formula)
solve(tree)

{'x1': 1}

# 2. Tutorial on SAT Solving (ROBDD)

A Binary Decision Tree (BDT) is a tree structure that is commonly used to represent logical formulas. In a BDT, the nodes of the tree represent variables, and the branches represent the values that these variables can take on. Specifically, the branch where its parent node evaluates to true is called the high branch, while the branch where its parent node evaluates to false is called the low branch. The high branch leads to the high child of a parent node, while the low branch leads to the low child of a parent. The leaf nodes of a BDT evaluate to either true or false, which are the outcomes of the test values of each variable. When using ROBDD for SAT solving, we follow the same logic structure and parser as introduced above. 

In [3]:
## import the robdd solver functions and methods
import sys  
sys.path.insert(0, '../bdd/')
import robdd_solver

# let's try an example
sat_formula = ["and", "x1", "x2"]
result_d = robdd_solver.solve(sat_formula, get_time=False, multiple=True)
print(result_d)

# a third example
sat_formula = ["and", "x1", ["not", ["or", "x2", "x1"]]]
result_d = robdd_solver.solve(sat_formula, get_time=False, multiple=True)
print(result_d)

# a simple example
sat_formula = "x1"
result_d = robdd_solver.solve(sat_formula, get_time=False, multiple=True)
print(result_d)

[{'x1': 1, 'x2': 1}]
UNSAT
[{'x1': 1}]


# 3. Tutorial on SMT Solving

This is a tutorial section on SMT solving using recursive backtracking and min-conflicts. Many codes in this tutorial can be found (together with references and acknowledgements) in the .py software version in the github: https://github.com/zhaoy37/SAT_Solver. The user can also follow the readme tutorial on the github. We leave any evaluations and solver comparisons to the python version part of the project in case the user is interested.

## Background on SMT Solving

The SMT problem is similar to the SAT Problem with the exception that the formulas are many-sorted. In our project, we only consider the SMT problem over integer predicates with no support for arithmetic operators that are not in the set of $=, \le, \ge, <, >, \neq, +, -, *, //$ because this representation scheme is all we need to solve the problems in the /problems folder. However, such scheme can be easily expanded to a broader set of operators and predicate domains.

One example of an SMT problem is: $(y1 \le 2) \wedge (y2 = 3)$. This SMT formula is satisifed by $\{"y1" : 0, "y2" : 3\}$. We will create a function **solve_SMT** to solve SMT problems. The function accepts 5 parameters as a representation of the SMT formula to be solved:

\\

**sat_formula**: The SAT encoding of the SMT clauses. Semantically, each atom in the encoding maps to one SMT subclause (True and False are not used in sat_formula for SMT encoding). For instance, for the formula $(y1 \le 2) \wedge (y2 = 3)$, the sat_formula parameter is ["and", "x1", "x2"], where "x1" and "x2" map to the two SMT subclauses $(y1 \le 2)$ and $(y2 = 3)$ respectively. The syntax follows the following BNF:
$<sat\_formula> := <atom> | ["and", <sat\_formula>, <sat\_formula>] | ["or", <sat\_formula>, <sat\_formula>] | ["not", <sat\_formula>]$

$<atom> := r"x[0-9]+"$

\\

**encodings**: The SMT encoding is a dictionary mapping each atom from the SAT encoding to an SMT clause. For the formula $(y1 \le 2) \wedge (y2 = 3)$, the SMT encoding is $\{"x1" : ["le", "y1", 2], "x2" : ["eq", "y2", 3]\}$. The syntax follows the following BNF:

$<smt\_formula> := [<operator>, <expression>, <expression>]$

$<operator> := "le" | "ge" | "eq" | "lt" | "gt" | "nq"$

$<expression>$ is any integer or string with mathematical expressions with allowed operators without using r'x[0-9]\*' or r'X[0-9]\*'.

Semantically, "le" stands for $\le$, "ge" stands for $\ge$, "lt" stands for <, "gt" stands for >, "eq" stands for =, and "nq" stands for $\neq$. The SMT formula $[<operator>, <expression1>, <expression2>]$ represents $<expression1> <operator> <expression2>$. For example, ["lt", "y1", 2] means y1 < 2, and ["nq", "y1 + 3 * y2", "y2"] means $y1 + 3 * y2 \neq y2$.

\\

**smt_vars**: This is the list of all the variables used in the SMT encoding. For the example of $(y1 \le 2)$ and $(y2 = 3)$, this would be ["y1", "y2"].

\\

**lowerbound**: Semantically, this is the lower bound of the search space (the lower bound of that any SMT variable can be assigned to). If the solution is outside of the search space, we do not guarantee the correctness.

\\

**upperbound**: Semantically, this is the upper bound of the search space (the upper bound of that any SMT variable can be assigned to). If the solution is outside of the search space, we do not guarantee the correctness.

\\

As an example, to solve $(y1 \le 2) \wedge (y2 = 3)$ with the bounds $0 \le y1, y2 \le 10$, call the function like this: solve_SMT(["and", "x1", "x2"], {"x1": ["le", "y1", 2], "x2": ["eq", "y2", 3]}, ["y1", "y2"], 0, 10).

To solve $(y1 - 2 = y2) \wedge (y2 + y1 > 5)$ with the bounds $0 \le y1, y2 \le 10$, call the function like this: solve_SMT(["and", "x1", "x2"], {"x1": ["eq", "y1 - 2", "y2"], "x2": ["gt", "y2 + y1", 5]}, ["y1", "y2"], 0, 10).

Let's first write the kernel of solving using recursive backtracking. The solver kernel uses recursive backtracking to search through the search space of possible assignments, the space of the close intervals from the preselected lowerbound to the upperbound for each SMT variable. In the aforementioned example execution, the algorithm will try the sequence of assignnments {"y1" : 0, "y2" : 0}, {"y1" : 0, "y2" : 1} ... until {"x1": True, "x2": True} is the mapping{"x1" : ["le", "y1", 2], "x2" : ["lt", "y2", 1]} -> {"x1": True, "x2": True} is satisfied. The solver concludes unsatisfiability when the search space runs out for all possible solutions to the SAT encoding.

In [11]:
from google.colab import drive
import os

!pip install ply
drive.mount("/content/drive")
os.chdir("/content/drive/MyDrive/EduSAT_Interactive_Tutorial")
# We will use a function from a python file. Please edit the previous line to match up with your own directory.
from calculator import calculate

def evaluate_assignment(converted, assignment):
    """
    This is a function to evaluate an assignment given a converted formula:

    converted: This is the converted SMT encoding where each clause have to be true in the SAT encoding.
    The user is encouraged to jump to the solve_smt function for a better understanding.

    We will provide a function to generate the "converted" data strucutre shortly.
    For now, assume that all clauses in "converted" must be true for the SMT clause to be satisfied.

    assignment: an assignment to the formula.
    """
    for formula in converted:
        # Realize the variables.
        try:
          # We use a calculator that we built to calculate the result of a subclause given an assignment.
            var1 = calculate(formula[1], assignment)
            var2 = calculate(formula[2], assignment)
        except ZeroDivisionError:
            return False

        operator = formula[0]
        # Perform checking.
        if operator == "lt":
            if var1 >= var2:
                return False
        elif operator == "gt":
            if var1 <= var2:
                return False
        elif operator == "ge":
            if var1 < var2:
                return False
        elif operator == "le":
            if var1 > var2:
                return False
        elif operator == "eq":
            if var1 != var2:
                return False
        else:
            if var1 == var2:
                return False
    return True

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# This is the kernel to the SMT solver.
def solve_SMT_kernel(converted, smt_vars, lowerbound, upperbound, cur_assignment):
    """
    converted: This is the converted SMT encoding where each clause have to be true in the SAT encoding.
    smt_vars: A list of SMT variables.
    lowerbound: The lower bound of the search space.
    upperbound: the upper bound of the search space.
    cur_assignment: The current assignment in the recursive backtracking process.
    """
    if len(smt_vars) == len(cur_assignment):
        choice_assignment = dict()
        for i in range(len(smt_vars)):
            choice_assignment[smt_vars[i]] = cur_assignment[i]
        return evaluate_assignment(converted, choice_assignment), choice_assignment
    else:
        for choice in range(lowerbound, upperbound + 1):
            temp_assignment = cur_assignment.copy()
            temp_assignment.append(choice)
            success, solution = solve_SMT_kernel(converted, smt_vars, lowerbound, upperbound, temp_assignment)
            if success:
                return success, solution
        # We return False and an empty solution if UNSAT.
        return False, {}

In [13]:
# Let's try an example on the kernel.
solve_SMT_kernel([["le", "y1", 2],["ge", "y2 + y1", 10]], ["y1", "y2"], 0, 10, [])

(True, {'y1': 0, 'y2': 10})

Now, let's try the minconflicts method. The kernel uses the min-conflicts algorithm commonly used for solving Constraint Satisfaction Problem. The algorithm first randomly initializes the values (given the bounds) for the set of all SMT variables. While the assignment does not satisfy the formula, the algorithm randomly pick a variable from all the set of SMT variables that result in conflict(s), and find the value that best reduces the conflicts associated with that variable to update the assignment (the value found is equivalent to the value that best reduces the conflicts in the existing state of the assignment). The algorithm terminates when an assignment is found to satisfy the SMT formula or when the maximum number of iteration steps is achieved. **Please note that "UNSAT" may be found even if there exists a solution if the maximum iteration steps is not sufficiently large.** However, this approach can be faster than the naive backtracking approach when applied to some of our NP-complete problem solver(s) such as the N-queens problem solver.

In [14]:
import random

# I write a helper function to check if s is an integer (represented in string type).
def check_int(s):
    # I used the codes from here: https://stackoverflow.com/questions/1265665/how-can-i-check-if-a-string-represents-an-int-without-using-try-except
    if s[0] in ('-', '+'):
        return s[1:].isdigit()
    return s.isdigit()

def find_num_conflicts(converted, assignment):
    # This function calculates the total number of conflicts instead of the conflicts caused
    # by the variable of interest (The argmin should be equivalent).
    num_conflicts = 0
    for formula in converted:
        conflict_flag = False
        # Realize the variables.
        try:
            var1 = calculate(formula[1], assignment)
            var2 = calculate(formula[2], assignment)
        except ZeroDivisionError:
            num_conflicts += 1
            continue
        operator = formula[0]

        # Perform checking.
        if operator == "lt":
            if var1 >= var2:
                conflict_flag =  True
        elif operator == "gt":
            if var1 <= var2:
                conflict_flag =  True
        elif operator == "ge":
            if var1 < var2:
                conflict_flag =  True
        elif operator == "le":
            if var1 > var2:
                conflict_flag =  True
        elif operator == "eq":
            if var1 != var2:
                conflict_flag =  True
        else:
            if var1 == var2:
                conflict_flag =  True
        num_conflicts += conflict_flag
    return num_conflicts


def find_conflicted_variables(converted, assignment):
    conflicted = set()

    # This function is used for the min-conflicts kernel and find the variables in conflicts.
    for formula in converted:
        conflict_flag = False
        try:
            var1 = calculate(formula[1], assignment)
            var2 = calculate(formula[2], assignment)
        except ZeroDivisionError:
            vars_of_interest = []
            if type(formula[1]) == str:
                vars_of_interest.extend(formula[1].split())
            if type(formula[2]) == str:
                vars_of_interest.extend(formula[2].split())

            for var in vars_of_interest:
                if (var not in ["+", "-", "*", "/", "//"]) and (not check_int(var)):
                    conflicted.add(var)
            continue

        operator = formula[0]

        # Perform checking.
        if operator == "lt":
            if var1 >= var2:
                conflict_flag = True
        elif operator == "gt":
            if var1 <= var2:
                conflict_flag = True
        elif operator == "ge":
            if var1 < var2:
                conflict_flag = True
        elif operator == "le":
            if var1 > var2:
                conflict_flag = True
        elif operator == "eq":
            if var1 != var2:
                conflict_flag = True
        else:
            if var1 == var2:
                conflict_flag = True

        # Adding in the variables if necessary.
        if conflict_flag:
            vars_of_interest = []
            if type(formula[1]) == str:
                vars_of_interest.extend(formula[1].split())
            if type(formula[2]) == str:
                vars_of_interest.extend(formula[2].split())

            for var in vars_of_interest:
                if (var not in ["+", "-", "*", "/", "//"]) and (not check_int(var)):
                    conflicted.add(var)

    return conflicted

# This is the kernel for solving SMT using min-conflicts from Constraint Satisfaction Problem.
# This algorithm can result in UNSAT even there exists a solution if the argument max_steps is too small.
def solve_SMT_minconflicts_kernel(converted, smt_vars, lowerbound, upperbound, max_steps = 100):
    # Initially, randomly assign values to smt_vars.
    cur_assignment = dict()
    for var in smt_vars:
        cur_assignment[var] = random.randint(lowerbound, upperbound)

    for i in range(max_steps):
        if evaluate_assignment(converted, cur_assignment):
            return True, cur_assignment
        # Find all conflicted variables.
        conflicted_vars = list(find_conflicted_variables(converted, cur_assignment))
        if len(conflicted_vars) == 0:
            # There is no conflicted variable but the assignment evaluates to False.
            return False, {}
        # Randomly choose one conflicted variable.
        rand_index = random.randint(0, len(conflicted_vars) - 1)
        conflicted_var = conflicted_vars[rand_index]
        # Search for the value that minimizes the number of conflicts.
        conflicts_values= dict()
        best_num_conflicts = float("inf")
        for value in range(lowerbound, upperbound + 1):
            temp_assignment = cur_assignment.copy()
            temp_assignment[conflicted_var] = value
            num_conflicts = find_num_conflicts(converted, temp_assignment)
            if num_conflicts < best_num_conflicts:
                best_num_conflicts = num_conflicts

            if num_conflicts not in conflicts_values:
                conflicts_values[num_conflicts] = [value]
            else:
                conflicts_values[num_conflicts].append(value)

        # Break any tie randomly.
        possible_values = conflicts_values[best_num_conflicts]
        rand_index = random.randint(0, len(possible_values) - 1)
        best_value = possible_values[rand_index]
        cur_assignment[conflicted_var] = best_value

    return False, {}

In [15]:
# Let's try an example on the kernel.
solve_SMT_minconflicts_kernel([["le", "y1", 2],["ge", "y2 + y1", 10]], ["y1", "y2"], 0, 10)

(True, {'y1': 1, 'y2': 9})

Now, let's organize the kernels in one function solve_SMT.

In [16]:
from solver import solve

# Currently, the SMT solver only supports single solution
# (because I only need 1 for the NP-complete problem to be solved).
# The users are encouraged to extend the SMT solver to allow multiple solutions.
def solve_SMT(sat_formula, encodings, smt_vars, lowerbound, upperbound, method = "minconflicts"):
    """
    The list of all possible methods include:

    1. backtracking: The naive backtracking approach with a DPLL SAT Solver.
    3. minconflicts (default): The min-conflicts solver for constraint satisfaction problem.
    """
    # First, solve the sat_formula.
    tree = Logic(sat_formula)
    # We will use the multiple solution solver from DPLL (which we will not present here for simplicity.)
    sat_solutions = solve(tree, multiple = True)

    if sat_solutions == "UNSAT":
        return "UNSAT"

    for sat_solution in sat_solutions:
        # Convert all negative clauses to positive ones.
        converted = []
        for sat_atom in encodings:
            encoding = encodings[sat_atom].copy()
            if sat_solution[sat_atom]:
                converted.append(encoding)
            else:
                if encoding[0] == "le":
                    encoding[0] = "gt"
                elif encoding[0] == "ge":
                    encoding[0] = "lt"
                elif encoding[0] == "gt":
                    encoding[0] = "le"
                elif encoding[0] == "lt":
                    encoding[0] = "ge"
                elif encoding[0] == "nq":
                    encoding[0] = "eq"
                else:
                    encoding[0] = "nq"
                converted.append(encoding)
        # Perform recursive descent (or other algorithms)
        cur_assignment = []
        if method == "backtracking" or method == "robdd":
            success, solution = solve_SMT_kernel(converted, smt_vars, lowerbound, upperbound, cur_assignment)
        else:
            success, solution = solve_SMT_minconflicts_kernel(converted, smt_vars, lowerbound, upperbound)
        if success:
            return solution

    return "UNSAT"

In [17]:
solve_SMT(["and", "x1", "x2"], {"x1": ["nq", "y1 - y2", "y1 + y2"], "x2": ["eq", "y2 + y1", "y1"]},
                              ["y1", "y2"], 0, 10)

'UNSAT'

In [18]:
solve_SMT(["and", "x1", "x2"], {"x1": ["nq", "y1 - y2", "y1 + y2"], "x2": ["eq", "y2 + y1", "y1"]},
                              ["y1", "y2"], 0, 10, method = "backtracking")

'UNSAT'

In [19]:
solve_SMT(["and", "x1", "x2"], {"x1": ["eq", "y1 - 2", "y2"], "x2": ["gt", "y2 + y1", 5]}, ["y1", "y2"], 0, 10)

{'y1': 6, 'y2': 4}

In [20]:
solve_SMT(["and", "x1", "x2"], {"x1": ["eq", "y1 - 2", "y2"], "x2": ["gt", "y2 + y1", 5]}, ["y1", "y2"], 0, 10, method = "backtracking")

{'y1': 4, 'y2': 2}

# Tutorial on using the SMT solver to solve NP-complete problems

We will proceed to introduce some interesting problems we solve using our created SMT solver.

## Graph Coloring

Graph coloring is an NP-complete problem. Specifically, we focus on vertex-coloring: Given an undirected graph and a number of colors allowed, assign each node in the graph a color such that there does not exist a pair of adjacent nodes with the same color. The default SMT kernel used for this solver is backtracking.

We create the solve_graph_coloring function. The two arguments are graph, which is an adjacency list representation of a graph, and num_colors, which denotes the maximum number of colors allowed to color the graph. The function returns the assignment of chromatic numbers to each node if the problem is solvable (if the node does not appear in the assignment, it can be any color within the set of all chromatic numbers). In the case of an unsolvable problem, the solver returns "UNSAT".

Internally, the solver constructs the SMT encoding in the following way: 1) Each SMT clause dictates that the chromatic number of a node cannot be equal to that of one of its adjacent node. 2) Form the SAT representation of the SMT clauses by taking conjunctions of the SMT clauses, which cover all possible edges of the graph. 3) Use the SMT solver to solve the encoded SMT problem.

In [21]:
def solve_graph_coloring(graph, num_colors, method='backtracking'):
    if num_colors > len(list(graph.keys())):
        raise Exception("The maximum number of colors cannot be larger than the number of nodes in the graph.")

    # Formulate the SMT representation:

    # First, formulate the SMT encoding:
    smt_variables = set()
    smt_encoding = dict()
    index = 0
    for node in graph:
        connections = graph[node]
        for connected_node in connections:
            smt_encoding["x" + str(index)] = ["nq", node, connected_node]
            smt_variables.add(node)
            smt_variables.add(connected_node)
            index += 1
    smt_variables = list(smt_variables)

    if len(smt_variables) == 0:
        raise Exception("The maximum number of SMT Variables cannot be 0 for graph coloring.")

    # Now, formulate the SAT encoding:
    sat_encoding = []
    if len(smt_encoding.keys()) == 1:
        sat_encoding = list(smt_encoding.keys())[0]
    else:
        for sat_node in smt_encoding:
            if len(sat_encoding) == 0:
                sat_encoding = sat_node
            else:
                sat_encoding = ["and", sat_node, sat_encoding]

    # Find the lower and upper bound.
    lower_bound = 0
    upper_bound = num_colors - 1

    return solve_SMT(sat_encoding, smt_encoding, smt_variables, lower_bound, upper_bound, method = method)

In [22]:
# Let's try some examples:

# Let the graph be:
graph = {
    "y1" : ["y2", "y3", "y4"],
    "y2": ["y3"],
    "y3": ["y2"],
    "y4": []
}

# There needs to be at least one connection.

# This graph is:
#        y1
#       / | \
#      y2 y4 |
#      |     |
#      ------y3

# Now, solve the problem with our solver.

# First, solve the problem with only 2 colors. This should return UNSAT:
print("Solving Graph Coloring Problem: Example 1 --> Expected: UNSAT")
print("Solution from our solver:", solve_graph_coloring(graph, 2))
print()

# Now, solve the problem with 3 colors. This is solvable.
print("Solving Graph Coloring Problem: Example 2 --> Expected: This should be solvable.")
print("Solution from our solver:", solve_graph_coloring(graph, 3))
print()

Solving Graph Coloring Problem: Example 1 --> Expected: UNSAT
Solution from our solver: UNSAT

Solving Graph Coloring Problem: Example 2 --> Expected: This should be solvable.
Solution from our solver: {'y3': 0, 'y1': 1, 'y4': 0, 'y2': 2}



## N-Queens Problem

In the N-queens problem, given an n by n sized chess board, the algorithm is asked to place n queens on the board such that no two queen attack each other. The default SMT kernel used for this problem is backtracking.

We write the solve_n_queens function. The parameter is num_queens, which represents the number of queens to be palced (which is equivalent to the length of a side of the square chess board). The function, if the problem is solvable, will return a matrix representing the board, where the 1s denote the placement of the queens and 0s denote empty positions. If the given problem is not solvable, the solver will return "UNSAT".

Inside the solver, each SMT variable represents a column, and the assignmnent to that variable represents the row index of the queen in that column. Then, in the SMT encodings, the solver is supported with representations that no two queen can be in the same row and no two queens can be in the same diagonal. The SAT encoding further abstracts the SMT encodings by connecting the SMT encodings with conjunctions. If the problem is solvable, with the solution, the algorithm transforms the solution into a matrix representation of the chess board.

In [23]:
import numpy as np

def solve_n_queens(num_queens, method='backtracking'):
    # For the queens to not attack each other, they must be already in different columns.
    # The algorithm needs to find the row position of each queen in different columns.

    # First, formulate the smt variables, which represent the row position of each queen.
    smt_vars = []
    for i in range(num_queens):
        smt_vars.append("y" + str(i))

    # Now, find the smt encoding.
    # Ensure that no variables are in the same row:
    smt_encoding = dict()
    index = 0
    for i in range(len(smt_vars)):
        for j in range(i + 1, len(smt_vars)):
            smt_encoding["x" + str(index)] = ["nq", smt_vars[i], smt_vars[j]]
            index += 1

    # Ensure that no variables are in the same diagonal.
    for i in range(len(smt_vars)):
        column_1 = int(smt_vars[i][1:])
        for j in range(i + 1, len(smt_vars)):
            column_2 = int(smt_vars[j][1:])
            smt_encoding["x" + str(index)] = ["nq", smt_vars[i] + " - " + smt_vars[j], column_1 - column_2]
            index += 1
            smt_encoding["x" + str(index)] = ["nq", smt_vars[i] + " - " + smt_vars[j], column_2 - column_1]
            index += 1

    # Now, formulate the SAT encoding:
    sat_encoding = []
    if len(smt_encoding.keys()) == 1:
        sat_encoding = list(smt_encoding.keys())[0]
    else:
        for sat_node in smt_encoding:
            if len(sat_encoding) == 0:
                sat_encoding = sat_node
            else:
                sat_encoding = ["and", sat_node, sat_encoding]

    # Find the lower and upper bound.
    lower_bound = 0
    upper_bound = num_queens - 1
    solution = solve_SMT(sat_encoding, smt_encoding, smt_vars, lower_bound, upper_bound, method = method)

    if solution == "UNSAT":
        return solution

    # Now, configure the board:
    # Use numpy array for better visualization.
    board = np.array([[0] * num_queens] * num_queens)
    for column_var in solution:
        column = int(column_var[1:])
        row = solution[column_var]
        board[row][column] = 1
    return board

In [24]:
# Let's try some examples:
print("Solving the N-Queens problem: Example 1 (N = 3) --> Expected: UNSAT")
print("Solution from the solver:")
print(solve_n_queens(3, method = "minconflicts"))
print()
print("Solving the N-Queens problem: Example 2 (N = 4) --> Expected: This should be solvable.")
print("Solution from the solver:")
print(solve_n_queens(4, method = "minconflicts"))
print()
print("Solving the N-Queens problem: Example 2 (N = 8) --> Expected: This should be solvable. It may take a while.")
print("Solution from the solver:")
print(solve_n_queens(8, method = "minconflicts"))
print()

Solving the N-Queens problem: Example 1 (N = 3) --> Expected: UNSAT
Solution from the solver:
UNSAT

Solving the N-Queens problem: Example 2 (N = 4) --> Expected: This should be solvable.
Solution from the solver:
[[0 0 1 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 1 0 0]]

Solving the N-Queens problem: Example 2 (N = 8) --> Expected: This should be solvable. It may take a while.
Solution from the solver:
[[0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0]]



## Subset Sum

Subset Sum, another NP-Complete problem, is stated as follows: Given a list, L, of positive integers, find the subset of the list that sum up to a given value, X. Using the SMT solver, we create a solver for Subset Sum. The default SMT kernel for this solver is backtracking.

We write the solve_subset_sum function. There are two required parameters: 1) target_list denotes the list of nonnegative integers, L. 2) target_sum denotes the target, X, of the subset to be summed up to. There are two optional parameters. When the solver returns "UNSAT", the algorithm reports that the problem is not feasible. Otherwise, upon a successful completion, the solver returns a dictionary mapping each index of the list L to a number 1 or 0, where 1 represents the presence of the variable given by that index in the solution subset and 0 represents the non-existence.

Internally, the solver encodes each index of the list to a variable, y_i, with potential values in {0, 1}. Then, the SMT_encoding denotes that sum(y_i * L[i]) == target_sum. The SAT_encoding connects the SMT representations via conjunctions.

In [25]:
def solve_subset_sum(target_list, target_sum, method='backtracking'):
    for value in target_list:
        if value <= 0:
            raise Exception("List element must be positive.")

    if len(target_list) == 0:
        return "UNSAT"

    if not isinstance(target_sum, int):
        raise Exception("Invalid type for target_sum.")

    # Formulate the SMT encoding.
    smt_variables = set()
    smt_encoding = dict()

    index = 0
    list_map = dict()
    # Now, ensure that the sum of the variables is the target sum.
    summation = ""
    for i in range(len(target_list)):
        list_var = f"y{i}"
        list_map[list_var] = i
        smt_variables.add(list_var)
        if i == len(target_list) - 1:
            summation += f"y{i} * {target_list[i]}"
        else:
            summation += f"y{i} * {target_list[i]}+ "
    smt_encoding["x" + str(index)] = ["eq", summation, target_sum]
    smt_variables = list(smt_variables)

    # Now, formulate the SAT encoding:
    sat_encoding = []
    if len(smt_encoding.keys()) == 1:
        sat_encoding = list(smt_encoding.keys())[0]
    else:
        for sat_node in smt_encoding:
            if len(sat_encoding) == 0:
                sat_encoding = sat_node
            else:
                sat_encoding = ["and", sat_node, sat_encoding]
    solution = solve_SMT(sat_encoding, smt_encoding, smt_variables, 0, 1, method = method)
    if solution == "UNSAT":
        return "UNSAT"

    final_solution = dict()
    for node in solution:
        final_solution[list_map[node]] = solution[node]

    return final_solution

In [26]:
# Let's try some examples:
print("Solving the subset sum problem: [1, 2] -> 1 --> Expected: SAT")
print(solve_subset_sum([1, 2], 1))
print()
print("Solving the subset sum problem: [1, 2] -> 3 --> Expected: SAT")
print(solve_subset_sum([1, 2], 3))
print()
print("Solving the subset sum problem: [1, 2] -> 4 --> Expected: UNSAT")
print(solve_subset_sum([1, 2], 4))
print()

Solving the subset sum problem: [1, 2] -> 1 --> Expected: SAT
{1: 0, 0: 1}

Solving the subset sum problem: [1, 2] -> 3 --> Expected: SAT
{1: 1, 0: 1}

Solving the subset sum problem: [1, 2] -> 4 --> Expected: UNSAT
UNSAT



## Independent Set

Independent Set is an NP-Complete Problem. We frame our variant of the Independent Set problem below:

Find a set A, a subset of V, in an undirected graph G = (V, E), where every node in A is not adjacent to any other node in A and the cardinality of A is k.

We also frame the maximum independent set problem as the problem to find A with largest possible k.

The default SMT kernel for this solver is backtracking.

We write the solve_independent_set function. There are two required parameters: 1) graph denotes the adjacency list representation of the undirected graph G. 2) target_cardinality denotes the cardinality, k, of the independent set, A. If the problem is solvable, the function will return a list of nodes, which is a candidate solution for A. Otherwise, "UNSAT" will be returned.

We also create a solver for the maximum independent set problem: We write the find_maximum_independent_set function. The only required parameter is graph, which has the same meaning as the graph in the independent set problem solver.

For the independent set problem solver, the algorithm treats each node to take either values of 0 and 1. The constraints are that the sum of the values of two adjacent nodes is less then 2, and the sum of all the nodes in the graph equals the target cardinality. The SMT representation is then solved using our solver for SMT problems. For the maximum independent set problem solver, the algorithm keeps calling the independent set problem solver with incrementally larger value of target cardinality with a incrementation of 1 until "UNSAT" is returned. Then, the algorithm returns the solution with target cardinality one smaller than the cardinality that triggers the infeasibility.

In [27]:
def solve_independent_set(graph, target_cardinality, method='backtracking'):

    # Check target_cardinality.
    if target_cardinality < 1:
        raise Exception("Target Cardinality must be at least 1.")

    # First, formulate the SMT encoding:
    smt_variables = set()
    smt_encoding = dict()
    index = 0
    for node in graph:
        connections = graph[node]
        smt_variables.add(node)
        for connected_node in connections:
            smt_encoding["x" + str(index)] = ["lt", node + " + " + connected_node, 2]
            smt_variables.add(connected_node)
            index += 1

    graph_nodes = list(graph.keys())
    if len(graph_nodes) == 0:
        raise Exception("Number of nodes in the graph cannot be 0.")

    # Assert that the sum is equal to target_cardinality.
    summation = ""
    for i in range(len(graph_nodes)):
        if i == len(graph_nodes) - 1:
            summation += graph_nodes[i]
        else:
            summation += (graph_nodes[i] + " + ")
    smt_encoding[summation] = ["eq", summation, target_cardinality]
    smt_variables = list(smt_variables)

    # Now, formulate the SAT encoding:
    sat_encoding = []
    if len(smt_encoding.keys()) == 1:
        sat_encoding = list(smt_encoding.keys())[0]
    else:
        for sat_node in smt_encoding:
            if len(sat_encoding) == 0:
                sat_encoding = sat_node
            else:
                sat_encoding = ["and", sat_node, sat_encoding]

    solution = solve_SMT(sat_encoding, smt_encoding, smt_variables, 0, 1, method = method)

    if solution == "UNSAT":
        return "UNSAT"

    # Summarize the answer.
    answer = []
    for node in solution:
        if node in graph and solution[node] == 1:
            answer.append(node)
    return answer


def find_maximum_independent_set(graph, method = 'backtracking'):
    i = 1
    solution = solve_independent_set(graph, i, method = method)
    prev_solution = solution
    while solution != "UNSAT":
        i += 1
        prev_solution = solution
        solution = solve_independent_set(graph, i, method=method)

    return prev_solution

In [28]:
# Let's try some examples:

# We reuse the graph from before:
# This graph is:
#        y1
#       / | \
#      y2 y4 |
#      |     |
#      ------y3

print("Solving independent set problem with cardinality 1 on the same graph for Problem 1. --> Expected: SAT")
print(solve_independent_set(graph, 1))
print("Solving independent set problem with cardinality 3 on the same graph for Problem 1. --> Expected: UNSAT")
print(solve_independent_set(graph, 3))
print("Finding the maximum independent set --> Expected: SAT")
print(find_maximum_independent_set(graph))
print()

Solving independent set problem with cardinality 1 on the same graph for Problem 1. --> Expected: SAT
['y2']
Solving independent set problem with cardinality 3 on the same graph for Problem 1. --> Expected: UNSAT
UNSAT
Finding the maximum independent set --> Expected: SAT
['y4', 'y2']



## Partition Problem

The Partition Problem is NP-complete, and is defined below for our solver implementation:

Given a list L, of integers, partition L into two sublists such that the sum of one sublist equals the sum of the other sublist.

We only focus on integer lists because the SMT solver we implement only work for integer predicates. The default SMT kernel for this solver is backtracking.

We write the solve_partition function. There is one required parameter, target_list, which is the list, L, to be analyzed. When the solver returns "UNSAT", the algorithm reports infeasibility. Otherwise, upon a successful completion, the solver returns two sublists that form the partition of the target list.

Internally, the solver encodes each index of the list to a variable (y_i) with a potential values in {0, 1}. Then, the SMT_encoding represents sum(y_i * L[i]) * 2 == sum(L[i]). Thus, All variables that have the same assigned value form one group. The algorithm treats the two groups as the partition of the target list. In our SMT encoding, we also dictate that the sum of all y_i cannot be 0 or the length of the target list to prevent the case where a group among the two groups is an empty set.

In [29]:
def solve_partition(target_list, method = 'backtracking'):

    if len(target_list) == 0:
        return "UNSAT"

    # Formulate the SMT encoding.
    smt_variables = set()
    smt_encoding = dict()

    # Dictate the range of each variable and generate the temporary variables.
    index = 0
    list_vars = dict()
    summation = ""
    for i in range(len(target_list)):
        smt_variables.add(f"y{i}")
        list_vars[f"y{i}"] = i
        if i == len(target_list) - 1:
            summation += (f"y{i} * {target_list[i]}")
        else:
            summation += (f"y{i} * {target_list[i]} + ")
    smt_encoding["x" + str(index)] = ["eq", f"({summation}) * 2", sum(target_list)]
    index += 1

    # Make sure that the sum is not 0 and is not the total cardinality.
    summation = ""
    for i in range(len(target_list)):
        if i == len(target_list) - 1:
            summation += f"y{i}"
        else:
            summation += f"y{i} + "
    smt_encoding["x" + str(index)] = ["nq", summation, 0]
    index += 1
    smt_encoding["x" + str(index)] = ["nq", summation, len(target_list)]
    smt_variables = list(smt_variables)

    # Now, formulate the SAT encoding:
    sat_encoding = []
    if len(smt_encoding.keys()) == 1:
        sat_encoding = list(smt_encoding.keys())[0]
    else:
        for sat_node in smt_encoding:
            if len(sat_encoding) == 0:
                sat_encoding = sat_node
            else:
                sat_encoding = ["and", sat_node, sat_encoding]

    solution = solve_SMT(sat_encoding, smt_encoding, smt_variables, 0, 1, method = method)

    if solution == "UNSAT":
        return solution

    group_0 = []
    group_1 = []
    for node in solution:
        if solution[node] == 0:
            group_0.append(target_list[list_vars[node]])
        else:
            group_1.append(target_list[list_vars[node]])

    return group_0, group_1

In [30]:
# Let's try some examples:
print("Solving the Partition Problem: [1, 5, 4] --> Expected: SAT")
print(solve_partition([1, 5, 4]))
print("Solving the Partition Problem: [3, 3] --> Expected: SAT")
print(solve_partition([3, 3]))
print("Solving the Partition Problem: [1, 2] --> Expected: UNSAT")
print(solve_partition([1, 2]))

Solving the Partition Problem: [1, 5, 4] --> Expected: SAT
([5], [4, 1])
Solving the Partition Problem: [3, 3] --> Expected: SAT
([3], [3])
Solving the Partition Problem: [1, 2] --> Expected: UNSAT
UNSAT


# Conclusion

In this tutorial, we introduced methods for SAT and SMT solving as well as how SMT solving can be applied to solving NP-complete problems. Please feel free to check out the python implementations, where we illustrate evaluations, testings, as well as natural language interfaces. If you have any questions or notice any mistake(s) in this tutorial, please feel free to email yiqi.zhao@vanderbilt.edu.